In [1]:
import pandas as pd
import numpy as np
import requests as requests
from pandas import to_datetime
import seaborn as sns
import matplotlib.pyplot as plt

# EOLIC Production Estimation

In [3]:
#First of all, we try to create a model for eolic energy with 2014 data that we can extrapolate afterwards
generation_2014 = pd.read_csv('./Data/Generation/Generation_2014.csv')

In [4]:
#We use info or head fucntions to take a quick view on data
generation_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6314 entries, 0 to 6313
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       6314 non-null   float64
 1   percentage  6314 non-null   float64
 2   datetime    6314 non-null   object 
 3   Type        6314 non-null   object 
dtypes: float64(2), object(2)
memory usage: 197.4+ KB


In [5]:
#In order to have a 'cleaner' datetime and to order it by datetime:

generation_2014['fecha'] = generation_2014['datetime'].str[:10]
generation_2014['fecha'] = pd.to_datetime(generation_2014['fecha'])
generation_2014 = generation_2014.sort_values(by = 'fecha', ignore_index = True)
generation_2014.head()

,value,percentage,datetime,Type,fecha
0,73065.0520,0.110822,2014-01-01T00:00:00.000+01:00,Hidráulica,2014-01-01
1,4616.7525,0.007003,2014-01-01T00:00:00.000+01:00,Residuos no renovables,2014-01-01
2,146015.5030,0.221470,2014-01-01T00:00:00.000+01:00,Nuclear,2014-01-01
3,54043.0770,0.081970,2014-01-01T00:00:00.000+01:00,Cogeneración,2014-01-01
4,25240.6550,0.038284,2014-01-01T00:00:00.000+01:00,Carbón,2014-01-01


In [7]:
generation_2014['Type'].unique()

array(['Hidráulica', 'Residuos no renovables', 'Nuclear', 'Cogeneración',
       'Carbón', 'Fuel + Gas', 'Otras renovables', 'Residuos renovables',
       'Motores diésel', 'Turbina de gas', 'Solar térmica',
       'Generación total', 'Turbina de vapor', 'Solar fotovoltaica',
       'Ciclo combinado', 'Turbinación bombeo', 'Eólica', 'Hidroeólica'],
      dtype=object)

In [8]:
#At this point we are only interested in eolic, therefore:
eolic_2014 = generation_2014[(generation_2014['Type'] == 'Eólica')]
eolic_2014.groupby('Type').describe()

value                                                              \
        count           mean          std        min      25%         50%   
Type                                                                        
Eólica  365.0  139813.783145  74102.74163  10360.619  80815.6  122554.358   

                               percentage                                \
               75%         max      count      mean       std       min   
Type                                                                      
Eólica  193319.156  350063.672      365.0  0.189508  0.094388  0.014651   

                                                
             25%       50%       75%       max  
Type                                            
Eólica  0.114097  0.172735  0.258653  0.437958

## Weather Data

In [73]:
#Let's prepare now the weather data
weather_2014 = pd.read_csv('./Data/Weather/Weather2014.csv')
weather_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92888 entries, 0 to 92887
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fecha        92888 non-null  object 
 1   indicativo   92888 non-null  object 
 2   nombre       92888 non-null  object 
 3   provincia    92888 non-null  object 
 4   altitud      92888 non-null  int64  
 5   tmed         89480 non-null  object 
 6   prec         88915 non-null  object 
 7   tmin         89484 non-null  object 
 8   horatmin     87598 non-null  object 
 9   tmax         89506 non-null  object 
 10  horatmax     87800 non-null  object 
 11  dir          83078 non-null  float64
 12  velmedia     84451 non-null  object 
 13  racha        83081 non-null  object 
 14  horaracha    83071 non-null  object 
 15  sol          49713 non-null  object 
 16  presMax      67974 non-null  object 
 17  horaPresMax  67970 non-null  object 
 18  presMin      67973 non-null  object 
 19  hora

In [69]:
#We downloaded also the metadata from the API request to a better understanding of the variables:
weather_metadata = pd.read_csv('./Data/Weather/Metadata.csv')
weather_metadata

,Unnamed: 0,id,descripcion,tipo_datos,unidad,requerido
0,0,fecha,fecha del dia (AAAA-MM-DD),string,NaN,True
1,0,indicativo,indicativo climatológico,string,NaN,True
2,0,nombre,nombre (ubicación) de la estación,string,NaN,True
3,0,provincia,provincia de la estación,string,NaN,True
4,0,altitud,altitud de la estación en m sobre el nivel del...,float,m,True
5,0,tmed,Temperatura media diaria,float,°C,False
6,0,prec,Precipitación diaria de 07 a 07,float,"mm (Ip = inferior a 0,1 mm) (Acum = Precipitac...",False
7,0,tmin,Temperatura Mínima del día,float,°C,False
8,0,horatmin,Hora y minuto de la temperatura mínima,string,UTC,False
9,0,tmax,Temperatura Máxima del día,float,°C,False


In [74]:
#There are some values in 'prec' assigned as 'Ip' when rain is almost 0.
#We are going to substitute those values - there are not many of them -  with 0

prec = weather_2014['prec'].str.contains('Ip')
precacum = weather_2014['prec'].str.contains('Acum')
prec.value_counts()


False    87343
True      1572
Name: prec, dtype: int64

In [75]:
#In order to replace 'Ip' values in 'prec':

weather_2014.loc[weather_2014['prec'] == 'Ip', 'prec'] = 0

In [76]:
#As we can see below there are not any more 'Ip', so we can proceed with the transformation of 'prec'
prec = weather_2014['prec'].str.contains('Ip')
precacum = weather_2014['prec'].str.contains('Acum')
prec.value_counts()

False    87343
Name: prec, dtype: int64

In [79]:
weather_2014_eolic['prec'].unique()

array(['0,0', '0,6', '4,6', '16,5', '0,5', nan, '15,8', '5,8', '1,6', 0,
       '1,2', '0,4', '10,8', '0,1', '2,5', '4,5', '5,5', '13,1', '2,2',
       '2,3', '0,2', '0,7', '5,0', '8,6', '0,8', '53,4', '3,2', '32,1',
       '3,4', '3,0', '5,3', '1,4', '6,2', '3,5', '14,6', '2,6', '2,8',
       '1,0', '4,8', '49,1', '9,5', '6,3', '26,2', '14,1', '11,8', '9,6',
       '7,7', '15,5', '14,9', '22,0', '12,2', '6,9', '1,5', '1,9', '8,8',
       '0,3', '8,7', '27,4', '2,0', '1,7', '3,6', '15,0', '11,1', '6,7',
       '3,8', '3,9', '27,0', '5,1', '7,3', '23,8', '5,7', '16,0', '16,6',
       '27,3', '35,3', '11,4', '3,7', '1,1', '19,4', '6,6', '1,8', '24,3',
       '19,2', '7,8', '11,0', '4,7', '9,2', '8,4', '15,7', '10,6', '4,4',
       '2,4', '21,0', '5,4', '5,9', '0,9', '17,0', '1,3', '14,0', '11,2',
       '25,2', '11,6', '29,0', '6,8', '4,3', '5,2', '3,1', '16,7', '6,0',
       '27,8', '28,2', '4,9', '8,5', '18,9', '23,4', '7,2', '13,0', '2,1',
       '6,1', '31,0', '12,1', '13,4', '28,7',

In [80]:
weather_2014_eolic = weather_2014[['fecha', 'nombre', 'provincia','prec','tmed', 'racha', 'velmedia']]
weather_2014_eolic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92888 entries, 0 to 92887
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      92888 non-null  object
 1   nombre     92888 non-null  object
 2   provincia  92888 non-null  object
 3   prec       88915 non-null  object
 4   tmed       89480 non-null  object
 5   racha      83081 non-null  object
 6   velmedia   84451 non-null  object
dtypes: object(7)
memory usage: 5.0+ MB


In [81]:
#We need to eliminate now NA values
weather_2014_eolic = weather_2014_eolic.dropna()
weather_2014_eolic.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 78944 entries, 0 to 92887
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      78944 non-null  object
 1   nombre     78944 non-null  object
 2   provincia  78944 non-null  object
 3   prec       78944 non-null  object
 4   tmed       78944 non-null  object
 5   racha      78944 non-null  object
 6   velmedia   78944 non-null  object
dtypes: object(7)
memory usage: 4.8+ MB


In [86]:
def transform_columns(columns, x):
    for column in columns:
        x[column] = x[column].str.replace(',', '.')
        x[column] = x[column].apply(pd.to_numeric)

In [87]:
cols = ['prec','tmed', 'racha', 'velmedia']
transform_columns(cols, weather_2014_eolic)
weather_2014_eolic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78944 entries, 0 to 92887
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fecha      78944 non-null  object 
 1   nombre     78944 non-null  object 
 2   provincia  78944 non-null  object 
 3   prec       77500 non-null  float64
 4   tmed       78944 non-null  float64
 5   racha      78944 non-null  float64
 6   velmedia   78944 non-null  float64
dtypes: float64(4), object(3)
memory usage: 4.8+ MB


In [89]:
#We calculate now the mean for the main variables that we are going to use in our calculations for each day
eolic_pivot_table = weather_2014_eolic.groupby(['fecha', 'provincia'], as_index=False)[['prec','tmed', 'racha', 'velmedia']].mean()
eolic_pivot_table = eolic_pivot_table.groupby('fecha', as_index = False)[['prec','tmed', 'racha', 'velmedia']].mean()
eolic_pivot_table

,fecha,prec,tmed,racha,velmedia
0,2014-01-01,4.167480,9.471099,11.786185,3.491249
1,2014-01-02,5.234907,11.461505,11.103541,3.162039
2,2014-01-03,5.125298,12.211928,10.751724,2.868819
3,2014-01-04,5.987120,9.503532,20.443871,5.478934
4,2014-01-05,1.909973,8.689526,12.754369,3.331294
...,...,...,...,...,...
360,2014-12-27,4.469773,7.959580,12.029483,3.253320
361,2014-12-28,1.504020,7.256511,14.356142,4.475836
362,2014-12-29,0.109873,4.450539,12.392415,3.505839
363,2014-12-30,0.021047,4.486318,8.524408,2.370174


In [90]:
#In order to be able to mix it with generation data, we need to adapt 'fecha' field
eolic_pivot_table['fecha']=eolic_pivot_table['fecha'].apply(to_datetime)

In [91]:
#Now it is time to mix the generation and weather data

eolic_model = pd.merge(eolic_2014 , eolic_pivot_table, how = 'inner', on = 'fecha')
eolic_model

,value,percentage,datetime,Type,fecha,prec,tmed,racha,velmedia
0,257896.205,0.391166,2014-01-01T00:00:00.000+01:00,Eólica,2014-01-01,4.167480,9.471099,11.786185,3.491249
1,270764.404,0.366209,2014-01-02T00:00:00.000+01:00,Eólica,2014-01-02,5.234907,11.461505,11.103541,3.162039
2,248822.078,0.332673,2014-01-03T00:00:00.000+01:00,Eólica,2014-01-03,5.125298,12.211928,10.751724,2.868819
3,315077.619,0.420168,2014-01-04T00:00:00.000+01:00,Eólica,2014-01-04,5.987120,9.503532,20.443871,5.478934
4,260674.588,0.375069,2014-01-05T00:00:00.000+01:00,Eólica,2014-01-05,1.909973,8.689526,12.754369,3.331294
...,...,...,...,...,...,...,...,...,...
360,230548.649,0.330679,2014-12-27T00:00:00.000+01:00,Eólica,2014-12-27,4.469773,7.959580,12.029483,3.253320
361,298721.125,0.412681,2014-12-28T00:00:00.000+01:00,Eólica,2014-12-28,1.504020,7.256511,14.356142,4.475836
362,233870.648,0.306219,2014-12-29T00:00:00.000+01:00,Eólica,2014-12-29,0.109873,4.450539,12.392415,3.505839
363,155414.492,0.207185,2014-12-30T00:00:00.000+01:00,Eólica,2014-12-30,0.021047,4.486318,8.524408,2.370174


# Data Visualization

In [92]:
eolic_model.describe()

,value,percentage,prec,tmed,racha,velmedia
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,139813.783145,0.189508,1.804480,15.447407,10.289648,2.900969
std,74102.741630,0.094388,2.337083,5.556500,2.406338,0.783089
min,10360.619000,0.014651,0.000000,4.450539,5.453845,1.303036
25%,80815.600000,0.114097,0.170962,10.207262,8.818386,2.442213
50%,122554.358000,0.172735,0.804047,15.881245,9.932908,2.780006
75%,193319.156000,0.258653,2.737183,20.284183,11.313565,3.331294
max,350063.672000,0.437958,17.325878,25.824761,20.443871,6.217559
